In [24]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

from gensim.models.phrases import Phrases, Phraser

import gensim
import multiprocessing

from gensim.models import Word2Vec

In [2]:
#  input file

df = pd.read_csv('../data/processed/author-parse-articles.csv')

# Getting Domains From News Websites 
m = df['art_article_url'].str.extract('(?<=http://)(.*?)(?=/)|(?<=https://)(.*?)(?=/)')
m = m[0].fillna(m[1]).fillna(df['art_article_url'])
df['art_extracted_domain'] = m
df.head()

,Unnamed: 0,art_Unnamed: 0,art_uid,art_article_author,art_article_date,art_article_text,art_article_url,ref_Unnamed: 0,ref_Unnamed: 0.1,ref_uid,...,ref_time,ref_source,ref_time-capture,ref_sc,ref_domain,ref_full-text,ref_sc2,ref_geo,ref_time_parsed,art_extracted_domain
0,0,5,02b5cecd-fac3-4b89-80a2-5bc7d3356d51,['Jake Epstein'],2022-04-04 00:00:00,Lithuania's top diplomat said buying Russian o...,https://www.businessinsider.com/lithuania-dipl...,16,16,02b5cecd-fac3-4b89-80a2-5bc7d3356d51,...,4 hours ago,https://twitter.com/idreesali114/status/151098...,2022-04-04 13:47:10.051481,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,54°41′N 25°16′E,2022-04-04 09:50:58,www.businessinsider.com
1,1,1,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,"['Martin Belam', 'Léonie Chao-Fong', 'Gloria O...",2022-04-04 00:00:00,"This live blog is now closed, you can follow a...",https://www.theguardian.com/world/live/2022/ap...,17,17,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,...,4 hours ago,https://twitter.com/prm_ua/status/151097769528...,2022-04-04 13:47:14.411519,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,50°38′N 29°55′E,2022-04-04 09:50:58,www.theguardian.com
2,2,2,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,['Writer Smith'],2022-04-04 17:59:18+00:00,The Prosecutor General’s Office was able to re...,https://news84media.com/russia-ukraine/the-wor...,17,17,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,...,4 hours ago,https://twitter.com/prm_ua/status/151097769528...,2022-04-04 13:47:14.411519,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,50°38′N 29°55′E,2022-04-04 09:50:58,news84media.com
3,3,3,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,['Scott Lucas'],2022-04-04 06:24:08+01:00,Larisa Savenko outside her damaged house in Bu...,https://eaworldview.com/2022/04/ukraine-war-bu...,17,17,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,...,4 hours ago,https://twitter.com/prm_ua/status/151097769528...,2022-04-04 13:47:14.411519,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,50°38′N 29°55′E,2022-04-04 09:50:58,eaworldview.com
4,4,4,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,"['Iryna Khalip', 'Natallia Radzina', 'Andrei S...",2022-04-04 00:00:00,AFU Repels Enemy Attacks On All Fronts 8 4.04....,https://charter97.org/en/news/2022/4/4/462037/,17,17,8ea2c08f-d9ff-4aa8-b710-9b6f26c7bb39,...,4 hours ago,https://twitter.com/prm_ua/status/151097769528...,2022-04-04 13:47:14.411519,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,50°38′N 29°55′E,2022-04-04 09:50:58,charter97.org


In [3]:
article = df.loc[df['art_extracted_domain']=='www.compuserve.com']

In [4]:
article.shape

(1, 21)

In [5]:
article

,Unnamed: 0,art_Unnamed: 0,art_uid,art_article_author,art_article_date,art_article_text,art_article_url,ref_Unnamed: 0,ref_Unnamed: 0.1,ref_uid,...,ref_time,ref_source,ref_time-capture,ref_sc,ref_domain,ref_full-text,ref_sc2,ref_geo,ref_time_parsed,art_extracted_domain
6342,6342,8,436ee9aa-945f-4ed3-9566-a0b8f0f018d8,['The Associated Press'],2022-04-06,Live Updates | Russia-Ukraine War\n\nWASHINGTO...,https://www.compuserve.com/pf/story/0001/20220...,14,14,436ee9aa-945f-4ed3-9566-a0b8f0f018d8,...,4 hours ago,https://twitter.com/shashj/status/151098248417...,2022-04-04 13:46:59.387389,False,"data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...",SourceOn live map\nTell friends\n4 hours ago -...,False,59°54′N 10°44′E,2022-04-04 09:50:58,www.compuserve.com


# Cleaning part

In [7]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) 
# disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [8]:
# remove non-alphabetic characters
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in article['art_article_text'])

In [12]:
# Taking advantage of spaCy .pipe() 
# attribute to speed-up the cleaning process:
t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.01 mins


In [48]:
#Put the results in a DataFrame 
#to remove missing values and duplicates:

df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(1, 1)

In [49]:
sent = [row.split() for row in df_clean['clean']]

In [50]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

INFO - 21:15:41: collecting all words and their counts
INFO - 21:15:41: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 21:15:41: collected 2800 token types (unigram + bigrams) from a corpus of 2145 words and 1 sentences
INFO - 21:15:41: merged Phrases<2800 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 21:15:41: Phrases lifecycle event {'msg': 'built Phrases<2800 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 0.02s', 'datetime': '2022-04-26T21:15:41.071598', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 07:03:16) \n[Clang 10.0.0 ]', 'platform': 'Darwin-21.4.0-x86_64-i386-64bit', 'event': 'created'}


In [51]:
bigram = gensim.models.Phrases(sent, min_count=2, threshold=100)
sentences = bigram[sent]

INFO - 21:15:43: collecting all words and their counts
INFO - 21:15:43: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 21:15:43: collected 2800 token types (unigram + bigrams) from a corpus of 2145 words and 1 sentences
INFO - 21:15:43: merged Phrases<2800 vocab, min_count=2, threshold=100, max_vocab_size=40000000>
INFO - 21:15:43: Phrases lifecycle event {'msg': 'built Phrases<2800 vocab, min_count=2, threshold=100, max_vocab_size=40000000> in 0.01s', 'datetime': '2022-04-26T21:15:43.921893', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 07:03:16) \n[Clang 10.0.0 ]', 'platform': 'Darwin-21.4.0-x86_64-i386-64bit', 'event': 'created'}


In [52]:
bigram

In [53]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

917

In [56]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['s',
 'russian',
 'say',
 'ukraine',
 'russia',
 'war',
 'u',
 'wednesday',
 'president',
 'crime']

In [27]:
cores = multiprocessing.cpu_count() 
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

INFO - 21:07:10: Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2022-04-26T21:07:10.414819', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 07:03:16) \n[Clang 10.0.0 ]', 'platform': 'Darwin-21.4.0-x86_64-i386-64bit', 'event': 'created'}


# Building the Vocabulary Table:
Word2Vec requires us to build the vocabulary table (simply digesting all the words and filtering out the unique words, and doing some basic counts on them):

In [28]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 21:07:23: collecting all words and their counts
INFO - 21:07:23: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 21:07:23: collected 917 word types from a corpus of 2107 raw words and 1 sentences
INFO - 21:07:23: Creating a fresh vocabulary
INFO - 21:07:23: Word2Vec lifecycle event {'msg': 'effective_min_count=20 retains 9 unique words (0.9814612868047983%% of original 917, drops 908)', 'datetime': '2022-04-26T21:07:23.574206', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 07:03:16) \n[Clang 10.0.0 ]', 'platform': 'Darwin-21.4.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
INFO - 21:07:23: Word2Vec lifecycle event {'msg': 'effective_min_count=20 leaves 365 word corpus (17.323208353108686%% of original 2107, drops 1742)', 'datetime': '2022-04-26T21:07:23.574905', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 07:03:16) \n[Clang 10.0.0 ]', 'platform': 'Darwin-21.4.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
INFO - 21:07:23: 

Time to build vocab: 0.0 mins


# Training of the model:
Parameters of the training:

1. total_examples = int - Count of sentences;

2. epochs = int - Number of iterations (epochs) over the corpus - [10, 20, 30]

In [29]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 21:07:33: Word2Vec lifecycle event {'msg': 'training model with 7 workers on 9 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=20 window=2 shrink_windows=True', 'datetime': '2022-04-26T21:07:33.270169', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 07:03:16) \n[Clang 10.0.0 ]', 'platform': 'Darwin-21.4.0-x86_64-i386-64bit', 'event': 'train'}
INFO - 21:07:33: worker thread finished; awaiting finish of 6 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 5 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 4 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 3 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 2 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 1 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 0 more threads
INFO - 21:07:33: EPOCH - 1 : training on 2107 raw words (8 effective words) took 0.0s, 1209

INFO - 21:07:33: worker thread finished; awaiting finish of 3 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 2 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 1 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 0 more threads
INFO - 21:07:33: EPOCH - 13 : training on 2107 raw words (5 effective words) took 0.0s, 1242 effective words/s
INFO - 21:07:33: worker thread finished; awaiting finish of 6 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 5 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 4 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 3 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 2 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 1 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 0 more threads
INFO - 21:07:33: EPOCH - 14 : training on 2107 raw words (6 effe

INFO - 21:07:33: worker thread finished; awaiting finish of 4 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 3 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 2 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 1 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 0 more threads
INFO - 21:07:33: EPOCH - 26 : training on 2107 raw words (6 effective words) took 0.0s, 2148 effective words/s
INFO - 21:07:33: worker thread finished; awaiting finish of 6 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 5 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 4 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 3 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 2 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 1 more threads
INFO - 21:07:33: worker thread finished; awaiting finish of 0 mo

Time to train the model: 0.0 mins


# Similarity and Odd-One-Out:
Odd-One-Out:ask our model to give us the word that does not belong to the list!

In [30]:
w2v_model.wv.most_similar(positive=["war"])

[('s', 0.9992284774780273),
 ('russian', 0.9992067813873291),
 ('say', 0.9991176724433899),
 ('russia', 0.999078094959259),
 ('wednesday', 0.9990284442901611),
 ('u', 0.9990022778511047),
 ('president', 0.9986846446990967),
 ('ukraine', 0.9985914826393127)]

In [33]:
w2v_model.wv.most_similar(positive=["ukraine"])

[('s', 0.9986807703971863),
 ('say', 0.9985987544059753),
 ('war', 0.9985916018486023),
 ('russian', 0.9985873103141785),
 ('wednesday', 0.9985467791557312),
 ('u', 0.9984886050224304),
 ('russia', 0.998401403427124),
 ('president', 0.998171329498291)]

In [35]:
w2v_model.wv.most_similar(positive=["president"])

[('russian', 0.9987821578979492),
 ('s', 0.9987525939941406),
 ('war', 0.9986845254898071),
 ('say', 0.9986649751663208),
 ('u', 0.9986168146133423),
 ('wednesday', 0.9986019134521484),
 ('russia', 0.9985033273696899),
 ('ukraine', 0.9981712698936462)]

In [36]:
w2v_model.wv.similarity("president", 'war')

0.9986847

In [37]:
w2v_model.wv.similarity("u", 'war')

0.99900216

In [39]:
w2v_model.wv.doesnt_match(["u", "war", "russia"])

'u'